In [1]:
import os
from glob import glob

import numpy as np
from scipy.signal import butter, filtfilt, find_peaks


# ==========================
# 1. PATHS – EDIT THIS PART
# ==========================

KOCH_ROOT = r"C:\Users\RAZER\Documents\GitHub\Game_of_SQUIDE\SQUIDE\dATASET\dataset_2\koch"

MOESM6_DIR = os.path.join(KOCH_ROOT, "MOESM6")  # BSPM (ECG-like)
MOESM7_DIR = os.path.join(KOCH_ROOT, "MOESM7")  # part 1 of MCG (A1, V1, X1)
MOESM8_DIR = os.path.join(KOCH_ROOT, "MOESM8")  # part 2 of MCG (Z1, Z2, Z3)

OUT_FILE = "koch_pairs.npz"
FS = 1000  # sampling frequency: 1 ms interval


# ==========================
# 2. LOADING FUNCTIONS
# ==========================

def load_bspm_ecg(moesm6_dir):
    """
    Load BSPM (ECG-like) signals from MOESM6/BSPM_data/BSPM_CH_E*.txt

    Returns:
        ecg_bspm: (C_ecg, T)
        file_list: list of txt file paths
    """
    bspm_folder = os.path.join(moesm6_dir, "BSPM_data")
    txt_files = sorted(glob(os.path.join(bspm_folder, "BSPM_CH_E*.txt")))
    if not txt_files:
        raise RuntimeError(f"No BSPM_CH_E*.txt files found in {bspm_folder}")

    channels = []
    for f in txt_files:
        data = np.loadtxt(f, dtype=np.float32)
        channels.append(data)

    # Align lengths
    min_len = min(len(ch) for ch in channels)
    channels = [ch[:min_len] for ch in channels]
    ecg_bspm = np.stack(channels, axis=0)  # (C, T)

    print(f"[BSPM] Loaded {ecg_bspm.shape[0]} channels, length {ecg_bspm.shape[1]} samples")

    # Scale as per paper: multiply by -1.0 * 10^(-6) to get mV
    ecg_bspm = -1.0e-6 * ecg_bspm

    # Channel 33 is respiration → E33 → index 32 (0-based)
    if ecg_bspm.shape[0] >= 33:
        idx = np.arange(ecg_bspm.shape[0])
        idx = idx[idx != 32]
        ecg_bspm = ecg_bspm[idx, :]
        print(f"[BSPM] Dropped channel 33 (respiration). New shape: {ecg_bspm.shape}")

    return ecg_bspm, txt_files


def load_mcg_group(folder):
    """
    Load all *.txt files from a given MCG_CH_* folder, e.g. MCG_CH_A1, MCG_CH_Z1.

    Returns:
        arr: (C_group, T)
        files: list of txt paths
    """
    txt_files = sorted(glob(os.path.join(folder, "MCG_CH_*.txt")))
    if not txt_files:
        raise RuntimeError(f"No MCG_CH_*.txt in {folder}")

    channels = []
    for f in txt_files:
        data = np.loadtxt(f, dtype=np.float32)
        channels.append(data)

    min_len = min(len(ch) for ch in channels)
    channels = [ch[:min_len] for ch in channels]
    arr = np.stack(channels, axis=0)
    print(f"[MCG group] {folder} -> {arr.shape[0]} channels, {arr.shape[1]} samples")
    return arr, txt_files


def load_full_mcg(moesm7_dir, moesm8_dir):
    """
    Load all MCG channels from MOESM7 (MCG_CH_A1, V1, X1) and
    MOESM8 (MCG_CH_Z1, Z2, Z3) and concatenate along channel axis.

    Returns:
        mcg_full: (C_mcg_total, T)
        file_list: list of paths
    """
    # Known subfolders from your dir listing:
    mcg_groups_7 = ["MCG_CH_A1", "MCG_CH_V1", "MCG_CH_X1"]
    mcg_groups_8 = ["MCG_CH_Z1", "MCG_CH_Z2", "MCG_CH_Z3"]

    all_arrays = []
    all_files = []

    # Load A1, V1, X1
    for grp in mcg_groups_7:
        folder = os.path.join(moesm7_dir, grp)
        if os.path.isdir(folder):
            arr, files = load_mcg_group(folder)
            all_arrays.append(arr)
            all_files.extend(files)
        else:
            print(f"[WARN] Missing folder: {folder}")

    # Load Z1, Z2, Z3
    for grp in mcg_groups_8:
        folder = os.path.join(moesm8_dir, grp)
        if os.path.isdir(folder):
            arr, files = load_mcg_group(folder)
            all_arrays.append(arr)
            all_files.extend(files)
        else:
            print(f"[WARN] Missing folder: {folder}")

    if not all_arrays:
        raise RuntimeError("No MCG groups found in MOESM7 or MOESM8")

    # Align lengths across all groups
    min_len = min(a.shape[1] for a in all_arrays)
    all_arrays = [a[:, :min_len] for a in all_arrays]

    mcg_full = np.concatenate(all_arrays, axis=0)  # (C_total, T)
    print(f"[MCG] Total concatenated shape: {mcg_full.shape}")
    return mcg_full, all_files


# ==========================
# 3. R-PEAK DETECTION
# ==========================

def bandpass_filter(signal, fs=1000, low=5.0, high=15.0, order=2):
    nyq = 0.5 * fs
    b, a = butter(order, [low / nyq, high / nyq], btype="band")
    return filtfilt(b, a, signal)


def detect_r_peaks(signal, fs=1000, distance_ms=300, height_factor=0.4):
    """
    Very basic R-peak detector for clean signals.
    """
    filtered = bandpass_filter(signal, fs=fs, low=5.0, high=15.0, order=2)
    distance = int(distance_ms * fs / 1000.0)
    height = height_factor * np.max(filtered)
    peaks, _ = find_peaks(filtered, distance=distance, height=height)
    return peaks


def normalize_beat(x):
    """
    Channel-wise median/MAD normalization for a single beat window.
    x: (C, T)
    """
    m = np.median(x, axis=1, keepdims=True)
    mad = np.median(np.abs(x - m), axis=1, keepdims=True) + 1e-6
    return (x - m) / mad


# ==========================
# 4. MAIN
# ==========================

def main():
    # Load ECG (BSPM)
    ecg_bspm, ecg_files = load_bspm_ecg(MOESM6_DIR)

    # Load MCG (all groups from MOESM7 + MOESM8)
    mcg, mcg_files = load_full_mcg(MOESM7_DIR, MOESM8_DIR)

    # Align time length between ECG and MCG
    T = min(ecg_bspm.shape[1], mcg.shape[1])
    ecg_bspm = ecg_bspm[:, :T]
    mcg = mcg[:, :T]
    print(f"[ALIGN] ECG shape: {ecg_bspm.shape}, MCG shape: {mcg.shape}")

    # Choose one ECG channel for R-peaks (channel 0 for now)
    r_lead = ecg_bspm[0, :]
    r_peaks = detect_r_peaks(r_lead, fs=FS, distance_ms=300, height_factor=0.4)
    print(f"[R-PEAKS] Detected {len(r_peaks)} peaks")

    # Extract 2-second windows around each R-peak
    pre = int(0.5 * FS)   # 500 ms before R
    post = int(1.5 * FS)  # 1500 ms after R
    win_len = pre + post  # 2000 samples

    ecg_beats = []
    mcg_beats = []

    for r in r_peaks:
        start = r - pre
        end = r + post
        if start < 0 or end > T:
            continue

        ecg_win = ecg_bspm[:, start:end]  # (C_ecg, win_len)
        mcg_win = mcg[:, start:end]       # (C_mcg, win_len)

        if ecg_win.shape[1] != win_len or mcg_win.shape[1] != win_len:
            continue

        ecg_norm = normalize_beat(ecg_win).astype(np.float32)
        mcg_norm = normalize_beat(mcg_win).astype(np.float32)

        ecg_beats.append(ecg_norm)
        mcg_beats.append(mcg_norm)

    ecg_beats = np.stack(ecg_beats, axis=0)  # (N, C_ecg, 2000)
    mcg_beats = np.stack(mcg_beats, axis=0)  # (N, C_mcg, 2000)

    print(f"[RESULT] ECG beats: {ecg_beats.shape}, MCG beats: {mcg_beats.shape}")

    # Save to NPZ
    np.savez_compressed(
        OUT_FILE,
        ecg_beats=ecg_beats,
        mcg_beats=mcg_beats,
        fs=np.array([FS], dtype=np.int32),
        ecg_channel_files=np.array(ecg_files, dtype=object),
        mcg_channel_files=np.array(mcg_files, dtype=object),
        description=np.array(["Koch ECG (BSPM) + SQUID MCG, paired 2s beats"], dtype=object),
    )
    print(f"[SAVE] Saved Koch ECG–MCG pairs to {OUT_FILE}")


if __name__ == "__main__":
    main()


[BSPM] Loaded 33 channels, length 100000 samples
[BSPM] Dropped channel 33 (respiration). New shape: (32, 100000)
[MCG group] C:\Users\RAZER\Documents\GitHub\Game_of_SQUIDE\SQUIDE\dATASET\dataset_2\koch\MOESM7\MCG_CH_A1 -> 17 channels, 100000 samples
[MCG group] C:\Users\RAZER\Documents\GitHub\Game_of_SQUIDE\SQUIDE\dATASET\dataset_2\koch\MOESM7\MCG_CH_V1 -> 19 channels, 100000 samples
[MCG group] C:\Users\RAZER\Documents\GitHub\Game_of_SQUIDE\SQUIDE\dATASET\dataset_2\koch\MOESM7\MCG_CH_X1 -> 15 channels, 100000 samples
[MCG group] C:\Users\RAZER\Documents\GitHub\Game_of_SQUIDE\SQUIDE\dATASET\dataset_2\koch\MOESM8\MCG_CH_Z1 -> 17 channels, 100000 samples
[MCG group] C:\Users\RAZER\Documents\GitHub\Game_of_SQUIDE\SQUIDE\dATASET\dataset_2\koch\MOESM8\MCG_CH_Z2 -> 16 channels, 100000 samples
[MCG group] C:\Users\RAZER\Documents\GitHub\Game_of_SQUIDE\SQUIDE\dATASET\dataset_2\koch\MOESM8\MCG_CH_Z3 -> 16 channels, 100000 samples
[MCG] Total concatenated shape: (100, 100000)
[ALIGN] ECG shape:

In [2]:
import numpy as np

data = np.load("koch_pairs.npz", allow_pickle=True)
ecg_beats = data["ecg_beats"]
mcg_beats = data["mcg_beats"]

print("ECG beats:", ecg_beats.shape)  # (127, 32, 2000)
print("MCG beats:", mcg_beats.shape)  # (127, 100, 2000)
print("fs:", data["fs"])
print("desc:", data["description"])


ECG beats: (127, 32, 2000)
MCG beats: (127, 100, 2000)
fs: [1000]
desc: ['Koch ECG (BSPM) + SQUID MCG, paired 2s beats']


In [6]:
import numpy as np
import torch
from torch.utils.data import Dataset


class KochPairedBeatsDataset(Dataset):
    def __init__(self, npz_path="koch_pairs.npz", augment=False):
        """
        npz_path: path to the koch_pairs.npz file
        augment: if True, apply very light noise / jitter augmentations
        """
        data = np.load(npz_path, allow_pickle=True)
        self.ecg_beats = data["ecg_beats"]  # (N, C_ecg, T)
        self.mcg_beats = data["mcg_beats"]  # (N, C_mcg, T)
        self.fs = int(data["fs"][0])
        self.augment = augment

    def __len__(self):
        return self.ecg_beats.shape[0]

    def _augment(self, x):
        """
        x: numpy array (C, T)
        very light augmentations: small Gaussian noise and tiny time shift
        """
        # small noise
        x = x + 0.01 * np.random.randn(*x.shape).astype(np.float32)

        # tiny circular time shift up to ±20 samples
        max_shift = 20
        shift = np.random.randint(-max_shift, max_shift + 1)
        if shift != 0:
            x = np.roll(x, shift, axis=1)

        return x

    def __getitem__(self, idx):
        ecg = self.ecg_beats[idx]  # (C_ecg, T)
        mcg = self.mcg_beats[idx]  # (C_mcg, T)

        if self.augment:
            ecg = self._augment(ecg)
            mcg = self._augment(mcg)

        # convert to torch tensors
        ecg = torch.from_numpy(ecg)  # float32, shape (C_ecg, T)
        mcg = torch.from_numpy(mcg)  # float32, shape (C_mcg, T)

        return ecg, mcg


In [7]:
#from koch_dataset import KochPairedBeatsDataset
from torch.utils.data import DataLoader

ds = KochPairedBeatsDataset("koch_pairs.npz", augment=False)
print("Num beats:", len(ds))

loader = DataLoader(ds, batch_size=8, shuffle=True)
ecg, mcg = next(iter(loader))
print("ECG batch shape:", ecg.shape)  # expect: torch.Size([8, 32, 2000])
print("MCG batch shape:", mcg.shape)  # expect: torch.Size([8, 100, 2000])


Num beats: 127
ECG batch shape: torch.Size([8, 32, 2000])
MCG batch shape: torch.Size([8, 100, 2000])


In [11]:
import os
from glob import glob
from collections import defaultdict

import numpy as np
import wfdb
from scipy.signal import butter, filtfilt, find_peaks


# ==========================
# 1. CONFIG – EDIT THIS PATH
# ==========================

PTB_ROOT = r"C:\Users\RAZER\Documents\GitHub\Game_of_SQUIDE\SQUIDE\dATASET\physionet.org\files\ptbdb\1.0.0"   # TODO: change to your PTB folder
OUT_FILE = "ptb_beats.npz"

# Standard 12-lead names in PTB (lowercase in wfdb)
STANDARD_LEADS = ['i', 'ii', 'iii', 'avr', 'avl', 'avf',
                  'v1', 'v2', 'v3', 'v4', 'v5', 'v6']


# ==========================
# 2. HELPERS
# ==========================

def list_ptb_records(ptb_root):
    """
    Find all PTB records by searching for .hea files.
    Returns a list of record paths without extension, e.g.
    '...\\patient001\\s0010_re'
    """
    rec_paths = []
    for patient_dir in sorted(glob(os.path.join(ptb_root, 'patient*'))):
        for hea in sorted(glob(os.path.join(patient_dir, '*.hea'))):
            rec_paths.append(hea[:-4])  # drop '.hea'
    return rec_paths


def bandpass_filter(signal, fs=1000, low=5.0, high=15.0, order=2):
    """
    Simple bandpass filter for QRS detection.
    signal: 1D array
    """
    nyq = 0.5 * fs
    b, a = butter(order, [low / nyq, high / nyq], btype='band')
    return filtfilt(b, a, signal)


def detect_r_peaks(ecg_lead, fs=1000, distance_ms=300, height_factor=0.4):
    """
    Very simple R-peak detector on a single lead.
    ecg_lead: 1D numpy array
    """
    filtered = bandpass_filter(ecg_lead, fs=fs, low=5.0, high=15.0, order=2)
    distance = int(distance_ms * fs / 1000.0)   # minimal distance between peaks
    height = height_factor * np.max(filtered)   # threshold
    peaks, _ = find_peaks(filtered, distance=distance, height=height)
    return peaks


# ==========================
# 3. MAIN: BUILD PTB BEATS
# ==========================

def main():
    rec_paths = list_ptb_records(PTB_ROOT)
    print(f"Found {len(rec_paths)} PTB records")

    all_beats = []
    all_record_names = []
    all_subject_ids = []
    all_labels = []  # placeholder for diagnosis if you parse later

    for rec_path in rec_paths:
        rec_name = os.path.basename(rec_path)
        patient_id = os.path.basename(os.path.dirname(rec_path))

        try:
            record = wfdb.rdrecord(rec_path)
        except Exception as e:
            print(f"[WARN] Failed to read {rec_path}: {e}")
            continue

        sig = record.p_signal       # shape (T, n_channels)
        fs = int(record.fs)         # should be 1000
        ch_names = [ch.lower() for ch in record.sig_name]

        # Map 12 standard leads to indices
        lead_indices = []
        for ln in STANDARD_LEADS:
            if ln in ch_names:
                lead_indices.append(ch_names.index(ln))
            else:
                print(f"[WARN] Lead {ln} not found in {rec_path}, skipping record.")
                lead_indices = []
                break

        if not lead_indices:
            continue

        # Extract matrix of shape (12, T)
        ecg_12 = sig[:, lead_indices].T
        T = ecg_12.shape[1]

        # Use lead II for R-peak detection
        if 'ii' not in ch_names:
            print(f"[WARN] No lead II in {rec_path}, skipping.")
            continue
        lead2_index = ch_names.index('ii')
        lead2 = sig[:, lead2_index]

        # Detect R-peaks
        r_peaks = detect_r_peaks(lead2, fs=fs, distance_ms=300, height_factor=0.4)

        if len(r_peaks) < 5:
            print(f"[WARN] Few R-peaks ({len(r_peaks)}) in {rec_path}, skipping.")
            continue

        # 2-second window: [-0.5s, +1.5s] around each R → 2000 samples
        pre = int(0.5 * fs)    # 500 samples
        post = int(1.5 * fs)   # 1500 samples
        win_len = pre + post   # 2000

        for r in r_peaks:
            start = r - pre
            end = r + post
            if start < 0 or end > T:
                continue

            beat = ecg_12[:, start:end]  # (12, 2000)
            if beat.shape[1] != win_len:
                continue

            # Per-beat normalization (median + MAD)
            m = np.median(beat, axis=1, keepdims=True)
            mad = np.median(np.abs(beat - m), axis=1, keepdims=True) + 1e-6
            beat_norm = (beat - m) / mad

            all_beats.append(beat_norm.astype(np.float32))
            all_record_names.append(rec_name)
            all_subject_ids.append(patient_id)
            all_labels.append(0)  # placeholder

    if not all_beats:
        print("No beats collected. Check PTB_ROOT and data structure.")
        return

    all_beats = np.stack(all_beats, axis=0)  # (N, 12, 2000)
    all_record_names = np.array(all_record_names)
    all_subject_ids = np.array(all_subject_ids)
    all_labels = np.array(all_labels)

    print("PTB beats shape:", all_beats.shape)

    np.savez_compressed(
        OUT_FILE,
        beats=all_beats,
        record_names=all_record_names,
        subject_ids=all_subject_ids,
        labels=all_labels,
        fs=np.array([1000], dtype=np.int32),
        description=np.array(["PTB 12-lead beats, 2s windows around R-peaks"], dtype=object),
    )
    print(f"Saved PTB beats to {OUT_FILE}")


if __name__ == "__main__":
    main()


Found 3 PTB records
PTB beats shape: (362, 12, 2000)
Saved PTB beats to ptb_beats.npz


In [12]:
import numpy as np

ptb = np.load("ptb_beats.npz", allow_pickle=True)
print("beats:", ptb["beats"].shape)          # (N, 12, 2000)
print("fs:", ptb["fs"])
print("desc:", ptb["description"])
print("examples from records:", ptb["record_names"][:5])
print("subjects:", ptb["subject_ids"][:5])


beats: (362, 12, 2000)
fs: [1000]
desc: ['PTB 12-lead beats, 2s windows around R-peaks']
examples from records: ['s0010_re' 's0010_re' 's0010_re' 's0010_re' 's0010_re']
subjects: ['patient001' 'patient001' 'patient001' 'patient001' 'patient001']


In [13]:
import numpy as np
import torch
from torch.utils.data import Dataset


class PTBBeatsDataset(Dataset):
    def __init__(self, npz_path="ptb_beats.npz", augment=True):
        data = np.load(npz_path, allow_pickle=True)
        self.beats = data["beats"]        # (N, 12, 2000)
        self.fs = int(data["fs"][0])
        self.augment = augment

    def __len__(self):
        return self.beats.shape[0]

    def _augment(self, x):
        # x: (12, T)
        x = x + 0.01 * np.random.randn(*x.shape).astype(np.float32)  # noise
        max_shift = 20
        shift = np.random.randint(-max_shift, max_shift + 1)
        if shift != 0:
            x = np.roll(x, shift, axis=1)
        return x

    def __getitem__(self, idx):
        beat = self.beats[idx]
        if self.augment:
            beat = self._augment(beat)
        return torch.from_numpy(beat)


In [14]:
from torch.utils.data import DataLoader

ds_ptb = PTBBeatsDataset("ptb_beats.npz", augment=True)
loader_ptb = DataLoader(ds_ptb, batch_size=32, shuffle=True)

batch = next(iter(loader_ptb))
print(batch.shape)  # torch.Size([32, 12, 2000])


torch.Size([32, 12, 2000])


In [15]:
import numpy as np
import torch
from torch.utils.data import Dataset


class PTBBeatsDataset(Dataset):
    def __init__(self, npz_path="ptb_beats.npz", augment=True):
        data = np.load(npz_path, allow_pickle=True)
        self.beats = data["beats"]        # (N, 12, 2000)
        self.fs = int(data["fs"][0])
        self.augment = augment

    def __len__(self):
        return self.beats.shape[0]

    def _augment(self, x):
        """
        x: numpy array (12, T)
        Simple SimCLR-style augmentations:
        - small Gaussian noise
        - small circular time shift
        """
        x = x + 0.01 * np.random.randn(*x.shape).astype(np.float32)
        max_shift = 20
        shift = np.random.randint(-max_shift, max_shift + 1)
        if shift != 0:
            x = np.roll(x, shift, axis=1)
        return x

    def __getitem__(self, idx):
        x = self.beats[idx]  # (12, 2000)

        # For SimCLR we need TWO views of the same beat
        x1 = self._augment(x.copy())
        x2 = self._augment(x.copy())

        x1 = torch.from_numpy(x1)  # (12, 2000)
        x2 = torch.from_numpy(x2)

        return x1, x2


In [17]:
from torch.utils.data import DataLoader
#from ptb_dataset import PTBBeatsDataset

ds = PTBBeatsDataset("ptb_beats.npz", augment=True)
print("Num PTB beats:", len(ds))

loader = DataLoader(ds, batch_size=32, shuffle=True)
v1, v2 = next(iter(loader))
print(v1.shape, v2.shape)  # expect: [32, 12, 2000] [32, 12, 2000]


Num PTB beats: 362
torch.Size([32, 12, 2000]) torch.Size([32, 12, 2000])


In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class ECGEncoderCNN(nn.Module):
    """
    Simple 1D CNN encoder for ECG beats.
    Input: (B, C=12, T=2000)
    Output: (B, feat_dim)
    """
    def __init__(self, in_channels=12, feat_dim=256):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels, 64, kernel_size=7, stride=2, padding=3)
        self.bn1   = nn.BatchNorm1d(64)

        self.conv2 = nn.Conv1d(64, 128, kernel_size=5, stride=2, padding=2)
        self.bn2   = nn.BatchNorm1d(128)

        self.conv3 = nn.Conv1d(128, 256, kernel_size=5, stride=2, padding=2)
        self.bn3   = nn.BatchNorm1d(256)

        self.conv4 = nn.Conv1d(256, 256, kernel_size=3, stride=2, padding=1)
        self.bn4   = nn.BatchNorm1d(256)

        self.global_pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(256, feat_dim)

    def forward(self, x):
        """
        x: (B, C, T)
        """
        x = F.relu(self.bn1(self.conv1(x)))  # (B, 64, T/2)
        x = F.relu(self.bn2(self.conv2(x)))  # (B, 128, T/4)
        x = F.relu(self.bn3(self.conv3(x)))  # (B, 256, T/8)
        x = F.relu(self.bn4(self.conv4(x)))  # (B, 256, T/16)
        x = self.global_pool(x)              # (B, 256, 1)
        x = x.squeeze(-1)                    # (B, 256)
        x = self.fc(x)                       # (B, feat_dim)
        return x


class ProjectionHead(nn.Module):
    """
    2-layer MLP projection head for SimCLR.
    Input: (B, feat_dim)
    Output: (B, proj_dim)
    """
    def __init__(self, in_dim=256, proj_dim=128):
        super().__init__()
        self.fc1 = nn.Linear(in_dim, in_dim)
        self.fc2 = nn.Linear(in_dim, proj_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        # L2 normalize
        x = F.normalize(x, p=2, dim=-1)
        return x


In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class ECGEncoderCNN(nn.Module):
    """
    Simple 1D CNN encoder for ECG beats.
    Input: (B, C=12, T=2000)
    Output: (B, feat_dim)
    """
    def __init__(self, in_channels=12, feat_dim=256):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels, 64, kernel_size=7, stride=2, padding=3)
        self.bn1   = nn.BatchNorm1d(64)

        self.conv2 = nn.Conv1d(64, 128, kernel_size=5, stride=2, padding=2)
        self.bn2   = nn.BatchNorm1d(128)

        self.conv3 = nn.Conv1d(128, 256, kernel_size=5, stride=2, padding=2)
        self.bn3   = nn.BatchNorm1d(256)

        self.conv4 = nn.Conv1d(256, 256, kernel_size=3, stride=2, padding=1)
        self.bn4   = nn.BatchNorm1d(256)

        self.global_pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(256, feat_dim)

    def forward(self, x):
        """
        x: (B, C, T)
        """
        x = F.relu(self.bn1(self.conv1(x)))  # (B, 64, T/2)
        x = F.relu(self.bn2(self.conv2(x)))  # (B, 128, T/4)
        x = F.relu(self.bn3(self.conv3(x)))  # (B, 256, T/8)
        x = F.relu(self.bn4(self.conv4(x)))  # (B, 256, T/16)
        x = self.global_pool(x)              # (B, 256, 1)
        x = x.squeeze(-1)                    # (B, 256)
        x = self.fc(x)                       # (B, feat_dim)
        return x


class ProjectionHead(nn.Module):
    """
    2-layer MLP projection head for SimCLR.
    Input: (B, feat_dim)
    Output: (B, proj_dim)
    """
    def __init__(self, in_dim=256, proj_dim=128):
        super().__init__()
        self.fc1 = nn.Linear(in_dim, in_dim)
        self.fc2 = nn.Linear(in_dim, proj_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        # L2 normalize
        x = F.normalize(x, p=2, dim=-1)
        return x


In [20]:
import torch
import torch.nn.functional as F


def simclr_nt_xent_loss(z1, z2, temperature=0.1):
    """
    z1, z2: (B, D) normalized embeddings from two views of the same batch.
    Returns scalar loss.
    """
    assert z1.shape == z2.shape
    batch_size = z1.shape[0]

    z = torch.cat([z1, z2], dim=0)  # (2B, D)
    sim = F.cosine_similarity(z.unsqueeze(1), z.unsqueeze(0), dim=-1)  # (2B, 2B)

    # Mask to remove self-similarity
    self_mask = torch.eye(2 * batch_size, dtype=torch.bool, device=z.device)
    sim = sim / temperature

    # For each anchor i in 0..2B-1, define positives and negatives
    # Positives: (i, i+B) or (i, i-B) depending on which half
    labels = torch.arange(2 * batch_size, device=z.device)
    labels = (labels + batch_size) % (2 * batch_size)  # positive index for each anchor

    # For cross-entropy, we need logits (2B, 2B-1) and labels
    sim = sim.masked_fill(self_mask, -1e9)

    loss = F.cross_entropy(sim, labels)
    return loss


In [23]:
import torch
from torch.utils.data import DataLoader

#from ptb_dataset import PTBBeatsDataset
#from models import ECGEncoderCNN, ProjectionHead, simclr_nt_xent_loss


def train_ptb_simclr(
    npz_path="ptb_beats.npz",
    batch_size=64,
    lr=1e-3,
    epochs=50,
    device="cpu"
):
    # Dataset & DataLoader
    dataset = PTBBeatsDataset(npz_path=npz_path, augment=True)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0)

    # Model
    encoder = ECGEncoderCNN(in_channels=12, feat_dim=256)
    projector = ProjectionHead(in_dim=256, proj_dim=128)

    encoder.to(device)
    projector.to(device)

    optimizer = torch.optim.AdamW(
        list(encoder.parameters()) + list(projector.parameters()),
        lr=lr,
        weight_decay=1e-4,
    )

    encoder.train()
    projector.train()

    for epoch in range(1, epochs + 1):
        running_loss = 0.0
        for step, (x1, x2) in enumerate(loader):
            x1 = x1.to(device)  # (B, 12, 2000)
            x2 = x2.to(device)

            optimizer.zero_grad()

            # Forward both views through encoder + projector
            h1 = encoder(x1)          # (B, 256)
            h2 = encoder(x2)          # (B, 256)
            z1 = projector(h1)        # (B, 128)
            z2 = projector(h2)        # (B, 128)

            loss = simclr_nt_xent_loss(z1, z2, temperature=0.1)

            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        avg_loss = running_loss / (step + 1)
        print(f"Epoch {epoch:03d}/{epochs} - SimCLR loss: {avg_loss:.4f}")

    # Save encoder weights to reuse for Koch cross-modal training
    torch.save(encoder.state_dict(), "ecg_encoder_ptb_simclr.pth")
    print("Saved pretrained ECG encoder to ecg_encoder_ptb_simclr.pth")


if __name__ == "__main__":
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print("Using device:", device)
    train_ptb_simclr(
        npz_path="ptb_beats.npz",
        batch_size=64,
        lr=1e-3,
        epochs=50,
        device=device,
    )


Using device: cpu
Epoch 001/50 - SimCLR loss: 2.6459
Epoch 002/50 - SimCLR loss: 1.1079
Epoch 003/50 - SimCLR loss: 0.6489
Epoch 004/50 - SimCLR loss: 0.4174
Epoch 005/50 - SimCLR loss: 0.3391
Epoch 006/50 - SimCLR loss: 0.2616
Epoch 007/50 - SimCLR loss: 0.2033
Epoch 008/50 - SimCLR loss: 0.1476
Epoch 009/50 - SimCLR loss: 0.1429
Epoch 010/50 - SimCLR loss: 0.1264
Epoch 011/50 - SimCLR loss: 0.1099
Epoch 012/50 - SimCLR loss: 0.1040
Epoch 013/50 - SimCLR loss: 0.0866
Epoch 014/50 - SimCLR loss: 0.0873
Epoch 015/50 - SimCLR loss: 0.0799
Epoch 016/50 - SimCLR loss: 0.0804
Epoch 017/50 - SimCLR loss: 0.0584
Epoch 018/50 - SimCLR loss: 0.0531
Epoch 019/50 - SimCLR loss: 0.0608
Epoch 020/50 - SimCLR loss: 0.0563
Epoch 021/50 - SimCLR loss: 0.0483
Epoch 022/50 - SimCLR loss: 0.0445
Epoch 023/50 - SimCLR loss: 0.0454
Epoch 024/50 - SimCLR loss: 0.0452
Epoch 025/50 - SimCLR loss: 0.0440
Epoch 026/50 - SimCLR loss: 0.0354
Epoch 027/50 - SimCLR loss: 0.0380
Epoch 028/50 - SimCLR loss: 0.0328
Ep

In [24]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Conv1DEncoder(nn.Module):
    """
    Generic 1D CNN encoder for time series.
    Used for ECG (C=12 or 32) and MCG (C=100).
    Input: (B, C, T)
    Output: (B, feat_dim)
    """
    def __init__(self, in_channels, feat_dim=256):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels, 64, kernel_size=7, stride=2, padding=3)
        self.bn1   = nn.BatchNorm1d(64)

        self.conv2 = nn.Conv1d(64, 128, kernel_size=5, stride=2, padding=2)
        self.bn2   = nn.BatchNorm1d(128)

        self.conv3 = nn.Conv1d(128, 256, kernel_size=5, stride=2, padding=2)
        self.bn3   = nn.BatchNorm1d(256)

        self.conv4 = nn.Conv1d(256, 256, kernel_size=3, stride=2, padding=1)
        self.bn4   = nn.BatchNorm1d(256)

        self.global_pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(256, feat_dim)

    def forward(self, x):
        # x: (B, C, T)
        x = F.relu(self.bn1(self.conv1(x)))  # (B, 64, T/2)
        x = F.relu(self.bn2(self.conv2(x)))  # (B, 128, T/4)
        x = F.relu(self.bn3(self.conv3(x)))  # (B, 256, T/8)
        x = F.relu(self.bn4(self.conv4(x)))  # (B, 256, T/16)
        x = self.global_pool(x)              # (B, 256, 1)
        x = x.squeeze(-1)                    # (B, 256)
        x = self.fc(x)                       # (B, feat_dim)
        return x


class ProjectionHead(nn.Module):
    """
    2-layer MLP projection head.
    Input: (B, feat_dim)
    Output: (B, proj_dim) normalized
    """
    def __init__(self, in_dim=256, proj_dim=128):
        super().__init__()
        self.fc1 = nn.Linear(in_dim, in_dim)
        self.fc2 = nn.Linear(in_dim, proj_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = F.normalize(x, p=2, dim=-1)
        return x


def cross_modal_info_nce(z_e, z_m, temperature=0.1):
    """
    Cross-modal InfoNCE loss between ECG (z_e) and MCG (z_m).
    z_e: (B, D) ECG embeddings (L2-normalized)
    z_m: (B, D) MCG embeddings (L2-normalized)

    We compute similarity matrix S = z_e @ z_m^T / T
    and use symmetric loss: ECG→MCG and MCG→ECG.
    """
    assert z_e.shape == z_m.shape
    B, D = z_e.shape

    # cosine similarity (since both are normalized, dot = cos)
    logits = z_e @ z_m.T / temperature  # (B, B)

    targets = torch.arange(B, device=z_e.device)

    # ECG→MCG
    loss_e2m = F.cross_entropy(logits, targets)

    # MCG→ECG (transpose)
    loss_m2e = F.cross_entropy(logits.T, targets)

    loss = 0.5 * (loss_e2m + loss_m2e)
    return loss


In [26]:
import torch
from torch.utils.data import DataLoader

#from koch_dataset import KochPairedBeatsDataset
#from models import Conv1DEncoder, ProjectionHead, cross_modal_info_nce


def train_koch_crossmodal(
    npz_path="koch_pairs.npz",
    batch_size=8,
    lr=1e-3,
    epochs=100,
    device="cpu",
    pretrained_ecg_path=None
):
    # 1) Dataset & DataLoader
    dataset = KochPairedBeatsDataset(npz_path=npz_path, augment=True)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0)

    # 2) Encoders: ECG (32 channels) and MCG (100 channels)
    ecg_encoder = Conv1DEncoder(in_channels=32, feat_dim=256)
    mcg_encoder = Conv1DEncoder(in_channels=100, feat_dim=256)

    # Optionally load pretrained ECG weights (from PTB SimCLR)
    if pretrained_ecg_path is not None:
        print(f"Loading PTB-pretrained ECG encoder from {pretrained_ecg_path}")
        load_ptb_weights_into_koch_encoder(ecg_encoder, pretrained_ecg_path,
                                           in_channels_ptb=12, in_channels_koch=32)

    # Projection heads
    ecg_proj = ProjectionHead(in_dim=256, proj_dim=128)
    mcg_proj = ProjectionHead(in_dim=256, proj_dim=128)

    ecg_encoder.to(device)
    mcg_encoder.to(device)
    ecg_proj.to(device)
    mcg_proj.to(device)

    # 3) Optimizer
    # You can use smaller LR for pretrained ECG encoder if you want
    params = list(ecg_encoder.parameters()) + list(mcg_encoder.parameters()) \
             + list(ecg_proj.parameters()) + list(mcg_proj.parameters())

    optimizer = torch.optim.AdamW(params, lr=lr, weight_decay=1e-4)

    # 4) Training loop
    ecg_encoder.train()
    mcg_encoder.train()
    ecg_proj.train()
    mcg_proj.train()

    for epoch in range(1, epochs + 1):
        running_loss = 0.0

        for step, (ecg, mcg) in enumerate(loader):
            ecg = ecg.to(device)  # (B, 32, 2000)
            mcg = mcg.to(device)  # (B, 100, 2000)

            optimizer.zero_grad()

            # Encode
            h_e = ecg_encoder(ecg)    # (B, 256)
            h_m = mcg_encoder(mcg)    # (B, 256)

            # Project
            z_e = ecg_proj(h_e)       # (B, 128)
            z_m = mcg_proj(h_m)       # (B, 128)

            # Cross-modal InfoNCE
            loss = cross_modal_info_nce(z_e, z_m, temperature=0.1)

            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        avg_loss = running_loss / (step + 1)
        print(f"Epoch {epoch:03d}/{epochs} - Cross-modal loss: {avg_loss:.4f}")

    # 5) Save trained models
    torch.save(ecg_encoder.state_dict(), "ecg_encoder_koch.pth")
    torch.save(mcg_encoder.state_dict(), "mcg_encoder_koch.pth")
    torch.save(ecg_proj.state_dict(), "ecg_proj_koch.pth")
    torch.save(mcg_proj.state_dict(), "mcg_proj_koch.pth")
    print("Saved ECG + MCG encoders and projection heads for Koch dataset")

import torch

def load_ptb_weights_into_koch_encoder(ecg_encoder, ckpt_path, in_channels_ptb=12, in_channels_koch=32):
    """
    Load PTB-pretrained weights (12 channels) into a Koch ECG encoder (32 channels).
    All layers except conv1 are loaded directly.
    conv1.weight is adapted by tiling the 12-channel kernels to 32 channels.
    """
    ckpt = torch.load(ckpt_path, map_location="cpu")
    enc_state = ecg_encoder.state_dict()

    new_state = {}

    for k, v in enc_state.items():
        if k == "conv1.weight":
            if "conv1.weight" in ckpt:
                w_ptb = ckpt["conv1.weight"]  # (64, 12, 7)
                # tile channels: repeat along in_channel dim until >=32, then cut
                repeat_factor = (in_channels_koch + in_channels_ptb - 1) // in_channels_ptb
                w_tiled = w_ptb.repeat(1, repeat_factor, 1)  # (64, 12*repeat, 7)
                w_tiled = w_tiled[:, :in_channels_koch, :]   # (64, 32, 7)
                new_state[k] = w_tiled
            else:
                new_state[k] = v  # fallback: keep random init
        elif k == "conv1.bias":
            if "conv1.bias" in ckpt:
                new_state[k] = ckpt["conv1.bias"]
            else:
                new_state[k] = v
        else:
            # for all other layers, load if shape matches, else keep default
            if k in ckpt and ckpt[k].shape == v.shape:
                new_state[k] = ckpt[k]
            else:
                new_state[k] = v

    ecg_encoder.load_state_dict(new_state)
    print("Loaded PTB weights into Koch ECG encoder (with conv1 channel adaptation).")




if __name__ == "__main__":
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print("Using device:", device)

    # If you have PTB-pretrained encoder:
    pretrained_path = "ecg_encoder_ptb_simclr.pth"  # or None if you didn’t train it yet

    train_koch_crossmodal(
    npz_path="koch_pairs.npz",
    batch_size=8,
    lr=1e-3,
    epochs=100,
    device=device,
    pretrained_ecg_path="ecg_encoder_ptb_simclr.pth",
    )



Using device: cpu
Loading PTB-pretrained ECG encoder from ecg_encoder_ptb_simclr.pth
Loaded PTB weights into Koch ECG encoder (with conv1 channel adaptation).
Epoch 001/100 - Cross-modal loss: 2.1133
Epoch 002/100 - Cross-modal loss: 2.0697
Epoch 003/100 - Cross-modal loss: 2.0526
Epoch 004/100 - Cross-modal loss: 2.0576
Epoch 005/100 - Cross-modal loss: 2.0723
Epoch 006/100 - Cross-modal loss: 2.0715
Epoch 007/100 - Cross-modal loss: 2.0712
Epoch 008/100 - Cross-modal loss: 2.0711
Epoch 009/100 - Cross-modal loss: 2.0711
Epoch 010/100 - Cross-modal loss: 2.0710
Epoch 011/100 - Cross-modal loss: 2.0709
Epoch 012/100 - Cross-modal loss: 2.0707
Epoch 013/100 - Cross-modal loss: 2.0701
Epoch 014/100 - Cross-modal loss: 2.0655
Epoch 015/100 - Cross-modal loss: 2.0041
Epoch 016/100 - Cross-modal loss: 1.9330
Epoch 017/100 - Cross-modal loss: 1.9330
Epoch 018/100 - Cross-modal loss: 1.7987
Epoch 019/100 - Cross-modal loss: 1.6050
Epoch 020/100 - Cross-modal loss: 1.5144
Epoch 021/100 - Cross

# 1. PTB ECG – Prepare Beat-Level Dataset

In [ ]:
import os
from glob import glob

import numpy as np
import wfdb
from scipy.signal import find_peaks, butter, filtfilt


def list_ptb_records(ptb_root):
    """
    ptb_root: path to PTB root folder, e.g. 'data/ptbdb'
    Returns list of full record paths without extension, e.g. '.../patient001/s0010_re'
    """
    rec_paths = []
    for patient_dir in sorted(glob(os.path.join(ptb_root, 'patient*'))):
        for rec in sorted(glob(os.path.join(patient_dir, '*.hea'))):
            rec_paths.append(rec[:-4])  # strip '.hea'
    return rec_paths


def bandpass_filter(signal, fs=1000, low=5.0, high=15.0, order=2):
    """
    Simple bandpass filter for QRS detection.
    signal: 1D numpy array
    fs: sampling frequency
    """
    nyq = 0.5 * fs
    b, a = butter(order, [low / nyq, high / nyq], btype='band')
    return filtfilt(b, a, signal)
    

def detect_r_peaks(ecg_lead, fs=1000, distance_ms=300, height_factor=0.5):
    """
    Very basic R-peak detector on a single lead.
    ecg_lead: 1D numpy array
    distance_ms: minimum distance between peaks
    height_factor: relative threshold on peak height
    Returns indices of R-peaks.
    """
    filtered = bandpass_filter(ecg_lead, fs=fs, low=5.0, high=15.0)
    distance = int(distance_ms * fs / 1000.0)
    # rough threshold: height_factor * max
    height = height_factor * np.max(filtered)
    peaks, _ = find_peaks(filtered, distance=distance, height=height)
    return peaks


In [ ]:
import os
import numpy as np
import wfdb
from collections import defaultdict

PTB_ROOT = "data/ptbdb"    # TODO: change to your path
OUT_FILE = "ptb_beats.npz" # output

# standard 12 leads in PTB header
STANDARD_LEADS = ['i', 'ii', 'iii', 'avr', 'avl', 'avf',
                  'v1', 'v2', 'v3', 'v4', 'v5', 'v6']

all_beats = []
all_record_names = []
all_subject_ids = []
all_labels = []  # you can later fill with MI/non-MI, etc.

rec_paths = list_ptb_records(PTB_ROOT)
print(f"Found {len(rec_paths)} records")

for rec_path in rec_paths:
    try:
        record = wfdb.rdrecord(rec_path)
    except Exception as e:
        print(f"Failed to read {rec_path}: {e}")
        continue

    sig = record.p_signal  # shape (T, n_channels)
    fs = int(record.fs)    # should be 1000
    ch_names = [ch.lower() for ch in record.sig_name]

    # map lead names to indices
    lead_indices = []
    for ln in STANDARD_LEADS:
        if ln in ch_names:
            lead_indices.append(ch_names.index(ln))
        else:
            print(f"Lead {ln} not found in {rec_path}, skipping record.")
            lead_indices = []
            break

    if not lead_indices:
        continue

    ecg_12 = sig[:, lead_indices].T  # shape (12, T)
    T = ecg_12.shape[1]

    # use lead II for R-peak detection
    lead2_index = ch_names.index('ii')
    lead2 = sig[:, lead2_index]
    r_peaks = detect_r_peaks(lead2, fs=fs, distance_ms=300, height_factor=0.4)

    if len(r_peaks) < 5:
        print(f"Few R-peaks in {rec_path}, skipping.")
        continue

    # 2-second window: [-500ms, +1500ms] → 2000 samples
    pre = int(0.5 * fs)
    post = int(1.5 * fs)
    win_len = pre + post

    for r in r_peaks:
        start = r - pre
        end = r + post
        if start < 0 or end > T:
            continue
        beat = ecg_12[:, start:end]  # (12, 2000)
        if beat.shape[1] != win_len:
            continue

        # simple per-beat normalization (median + MAD)
        m = np.median(beat, axis=1, keepdims=True)
        mad = np.median(np.abs(beat - m), axis=1, keepdims=True) + 1e-6
        beat_norm = (beat - m) / mad

        all_beats.append(beat_norm.astype(np.float32))
        all_record_names.append(os.path.basename(rec_path))
        # subject id is patient folder name
        all_subject_ids.append(os.path.basename(os.path.dirname(rec_path)))
        all_labels.append(0)  # placeholder; you can parse .hea for diagnosis later

all_beats = np.stack(all_beats, axis=0)  # (N, 12, 2000)
all_record_names = np.array(all_record_names)
all_subject_ids = np.array(all_subject_ids)
all_labels = np.array(all_labels)

print("PTB beats shape:", all_beats.shape)

np.savez_compressed(
    OUT_FILE,
    beats=all_beats,
    record_names=all_record_names,
    subject_ids=all_subject_ids,
    labels=all_labels,
    fs=np.array([1000], dtype=np.int32),
    description=np.array(["PTB 12-lead beats, 2s windows around R-peaks"], dtype=object)
)
print(f"Saved PTB beats to {OUT_FILE}")


# 2. Koch ECG–MCG – Prepare Paired Beat Dataset